In [86]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [141]:
%matplotlib inline
import numpy as np
import numpy.random as nd
import pandas as pd
import math
import matplotlib.pyplot as plt

import os
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from IPython.display import display

import random

In [142]:
data_train = pd.read_csv('train_data.csv')
data_test = pd.read_csv('test_data.csv')

In [1]:
columns_keep = [
    'Home Rank', 'Away Rank', 
    'Home GDP per Person (dollars)', 'Home GDP (millions)', 'Home Population', 'Home Temp Diff', 'Home Rain Diff',  
    'Away GDP per Person (dollars)', 'Away GDP (millions)', 'Away Population', 'Away Temp Diff', 'Away Rain Diff', 
]

In [169]:
X_train = data_train[columns_keep]
y_train_home = data_train['Home Team Goals']
y_train_away = data_train['Away Team Goals']

X_test = data_test[columns_keep]
y_test_home = data_test['Home Team Goals']
y_test_away = data_test['Away Team Goals']

In [170]:
def get_results(y_predict_home, y_predict_away):
    res = []
    for i in range(len(y_predict_home)):
        if y_predict_home[i] > y_predict_away[i]:
            res.append('W')
        elif y_predict_home[i] < y_predict_away[i]:
            res.append('L')
            
        else:
            res.append('D')
    
    return np.array(res)

In [171]:
model_home = KNeighborsRegressor(n_neighbors=18)
model_away = KNeighborsRegressor(n_neighbors=18)

model_fit_home = model_home.fit(X_train, y_train_home)
model_fit_away = model_away.fit(X_train, y_train_away)


In [189]:
y_predict_home = model_fit_home.predict(X_test)
y_predict_away = model_fit_away.predict(X_test)

y_predict = get_results(y_predict_home, y_predict_away)

accuracy_score(y_test, y_predict)

0.484375

In [175]:
y_predict_away

array([0.94444444, 1.27777778, 1.83333333, 1.27777778, 0.83333333,
       0.61111111, 0.33333333, 1.11111111, 0.44444444, 1.72222222,
       1.22222222, 0.83333333, 1.05555556, 1.61111111, 0.72222222,
       1.22222222, 0.72222222, 1.5       , 0.83333333, 1.77777778,
       0.55555556, 1.44444444, 0.55555556, 0.33333333, 1.11111111,
       1.11111111, 1.11111111, 1.        , 1.38888889, 1.38888889,
       1.38888889, 1.38888889, 1.61111111, 1.16666667, 0.94444444,
       1.72222222, 1.22222222, 0.61111111, 0.77777778, 1.44444444,
       0.88888889, 0.94444444, 1.44444444, 0.77777778, 1.33333333,
       0.77777778, 0.66666667, 0.83333333, 1.16666667, 1.22222222,
       1.22222222, 0.94444444, 0.77777778, 1.27777778, 1.11111111,
       1.11111111, 0.94444444, 1.16666667, 0.61111111, 1.38888889,
       1.38888889, 0.94444444, 1.33333333, 0.77777778])

In [174]:
y_predict_home

array([1.33333333, 1.44444444, 1.        , 1.33333333, 1.5       ,
       1.33333333, 1.66666667, 0.88888889, 1.55555556, 1.        ,
       0.83333333, 1.5       , 1.11111111, 1.27777778, 1.33333333,
       1.66666667, 1.61111111, 1.27777778, 1.55555556, 1.5       ,
       1.33333333, 1.61111111, 1.27777778, 1.66666667, 1.55555556,
       1.22222222, 0.88888889, 1.44444444, 1.11111111, 1.11111111,
       1.11111111, 1.11111111, 1.05555556, 0.72222222, 1.44444444,
       1.61111111, 1.27777778, 1.61111111, 1.5       , 1.61111111,
       0.88888889, 0.88888889, 1.61111111, 0.83333333, 0.83333333,
       0.83333333, 1.44444444, 0.94444444, 0.72222222, 1.61111111,
       1.27777778, 1.44444444, 1.5       , 1.16666667, 0.72222222,
       1.72222222, 1.5       , 0.72222222, 1.61111111, 1.11111111,
       1.11111111, 1.33333333, 1.16666667, 1.        ])

In [61]:
home_win = X_test['Home Rank'] > X_test['Away Rank']
y_predict_base = home_win.replace({True: 'W', False: 'L'})

In [62]:
accuracy_score(y_test, y_predict_base)

0.3125

In [176]:
X_train = data_train[columns_keep]
y_train_1= data_train['Home Team Goals'] - data_train['Away Team Goals']


X_test = data_test[columns_keep]
y_test_1 = data_test['Home Team Goals'] - data_test['Away Team Goals']

In [190]:
model = KNeighborsRegressor(n_neighbors=18)
model_fit = model.fit(X_train, y_train_1)
predictions = model_fit.predict(X_test)

def get_results_2(predictions):
    res = []
    for i in range(len(predictions)):
        if predictions[i] > 0:
            res.append('W')
        elif predictions[i] < 0:
            res.append('L')
            
        else:
            res.append('D')
    
    return np.array(res)

pred_array = get_results_2(predictions)
accuracy_score(y_test, pred_array)

0.484375